In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from geopy import distance


## Data from zenodo archive

https://zenodo.org/record/7923702

In [16]:
january=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190101_20190131.csv', dtype={1: 'string'})
february=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190201_20190228.csv', dtype={1: 'string'})
march=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190301_20190331.csv', dtype={1: 'string'})
april=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190401_20190430.csv', dtype={1: 'string'})
may=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190501_20190531.csv', dtype={1: 'string'})
june=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190601_20190630.csv', dtype={1: 'string'})
july=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190701_20190731.csv', dtype={1: 'string'})
august=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190801_20190831.csv', dtype={1: 'string'})
september=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20190901_20190930.csv', dtype={1: 'string'})
october=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20191001_20191031.csv', dtype={1: 'string'})
november=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20191101_20191130.csv', dtype={1: 'string'})
december=pd.read_csv('D:/a.salgas/Documents/opensky_2019/flightlist_20191201_20191231.csv', dtype={1: 'string'})


opensky_2019=pd.concat([january,february,march,april,may,june,july,august,september,october,november,december], axis=0).reset_index()

In [17]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Utilities")

import open_sky



In [18]:
os_flights_2019=open_sky.FlightOpenSky()
os_flights_2019.df=opensky_2019 # ==> the file was extracted from opensky zenodo archive with a notebook that can be found in Utilities folder

## TODO rework on preprocess to remove (now) unecessary steps, BEFORE FULL RUN
os_flights_2019.preprocess_data()


46.08808512890124% of flights deleted after removing flights with no origin and destination. 16830310 Flights in the dataset


D:\AeroSCOPE\aeroscope/Utilities\open_sky.py:48: DtypeWarning: Columns (4,5,17,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  ac_ref = pd.read_csv(data_path, sep=',')


Size of df after_grouping: 2196763; number of flights: 16830310
Size of df after ac info: 2196763; number of flights 16830310
Size of df after arpt info: 2187238, number of flights: 16761488


In [19]:
os_df=os_flights_2019.df

os_df

,airline_iata,origin,dest,aircraft_type,n_flights,acft_icao,acft_class,seymour_proxy,origin_iata,origin_lon,origin_lat,origin_country,origin_continent,dest_iata,dest_lon,dest_lat,dest_country,dest_continent
0,AAF,DAAG,LFBO,A320,55,A320,NB,A320,ALG,3.214531,36.693886,DZ,AF,TLS,1.363820,43.629101,FR,EU
1,AAF,DAAG,LFHJ,A320,1,A320,NB,A320,ALG,3.214531,36.693886,DZ,AF,NaN,4.913610,45.654202,FR,EU
2,AAF,DAAG,LFLL,A320,179,A320,NB,A320,ALG,3.214531,36.693886,DZ,AF,LYS,5.081111,45.725556,FR,EU
3,AAF,DAAG,LFMI,A320,3,A320,NB,A320,ALG,3.214531,36.693886,DZ,AF,NaN,4.923840,43.522701,FR,EU
4,AAF,DAAG,LFML,A320,123,A320,NB,A320,ALG,3.214531,36.693886,DZ,AF,MRS,5.221424,43.439272,FR,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196758,bfx,LSZM,EDDN,PRM1,2,PRM1,PJ,zzz,NaN,9.064662,47.078353,CH,EU,NUE,11.078056,49.498699,DE,EU
2196759,bfx,LSZR,EDDN,PRM1,1,PRM1,PJ,zzz,ACH,9.560770,47.485001,CH,EU,NUE,11.078056,49.498699,DE,EU
2196760,bfx,LZIB,EDCT,PRM1,1,PRM1,PJ,zzz,BTS,17.212700,48.170200,SK,EU,NaN,12.536944,51.395000,DE,EU
2196761,bfx,UKKK,EDDN,PRM1,1,PRM1,PJ,zzz,IEV,30.451940,50.401940,UA,EU,NUE,11.078056,49.498699,DE,EU


In [20]:
fleet=pd.read_excel('data/planespotters_fleet.xlsx')
fleet['Seat Total']=fleet['Seat Total'].replace(0, np.nan)
fleet.dropna(subset=['Seat Total'], inplace=True)

# Adding the name/icao code match table
match=pd.read_csv('data/fleet_match.csv', sep=';')
fleet=fleet.merge(match, left_on='Aircraft Type', right_on='spotter_name', how='right').drop(columns='spotter_name')

aircraft_data=fleet.groupby(['Aircraft Type','ICAO_AC'])['Seat Total'].mean().reset_index().drop(columns='Aircraft Type')

In [21]:
aircraft_data.head()

,ICAO_AC,Seat Total
0,AT43,46.897119
1,AT45,46.897119
2,AT72,70.137580
3,AT75,70.137580
4,AT76,70.137580


In [22]:
os_df=os_df.merge(aircraft_data, left_on='acft_icao', right_on='ICAO_AC', how='left')
os_df['Seat Total']=os_df['n_flights']*os_df['Seat Total']
os_df=os_df.groupby(['origin_iata','origin_lon','origin_lat','origin_country','origin_continent','dest_iata','dest_lon','dest_lat','dest_country','dest_continent','airline_iata','acft_icao','acft_class','seymour_proxy'])[['Seat Total','n_flights']].sum().reset_index().rename(columns={'Seat Total':'seats'})

In [23]:
column_mapping = {
    'origin_iata': 'iata_departure',
    'dest_iata': 'iata_arrival',
    'origin_lon': 'departure_lon',
    'origin_lat': 'departure_lat',
    'origin_country': 'departure_country',
    'origin_continent': 'departure_continent',
    'dest_lon': 'arrival_lon',
    'dest_lat': 'arrival_lat',
    'dest_country': 'arrival_country',
    'dest_continent': 'arrival_continent'
}                          


os_df.rename(columns=column_mapping, inplace=True)
os_df['source']='OpenSky'

In [24]:
iata_icao_convert=pd.read_csv('data/iata_icao_airline.csv', sep=';')
iata_icao_convert=iata_icao_convert.sort_values(by=['IATA'], ascending=False).drop_duplicates(subset='ICAO', keep='first')

In [25]:
os_df=os_df.merge(iata_icao_convert[['IATA','ICAO']], left_on='airline_iata', right_on='ICAO', how='left')

In [26]:
os_df['IATA']=os_df['IATA'].fillna(os_df['airline_iata'])
os_df=os_df.drop(columns=['airline_iata','ICAO']).rename(columns={'IATA':'airline_iata'})

In [27]:
os_df.loc[:, "distance_km"] = os_df.progress_apply(
            lambda x: distance.distance((float(x.departure_lat), float(x.departure_lon)),
                                        (float(x.arrival_lat), float(x.arrival_lon))).km if not (pd.isna(x.departure_lat) or pd.isna(x.departure_lon)
                                                                                           or pd.isna(x.arrival_lon) or pd.isna(x.arrival_lat)) else 0, axis=1)


  0%|          | 0/856982 [00:00<?, ?it/s]

In [28]:
os_df
os_df['ask'] = os_df["distance_km"]*os_df["seats"]
os_df['rpk'] = os_df['ask']*0.824

Load factor source: https://www.icao.int/annual-report-2019/Pages/the-world-of-air-transport-in-2019.aspx

In [29]:
os_df['seats'].sum()

1577734998.5943277

## Fuel burn computation process

Problem: we do not have a seymour proxi for all the aircraft types neither do we have the seats associated to each aircrfat like in AeroSCOPE core usage. 
One could extend the proxi list, but it is not without increasing the risk of errors.  
Helicopters are particularly often in this case as shown below, followed py piston private and private jets and few turbo props.  
Fortunately, this is not the majority of the flights.

If the interested reader is willing to increase the proxy list for a particular use case, he is free to do so ;) 

In [30]:
os_df[os_df.seymour_proxy=='zzz'].groupby('acft_class')['n_flights'].sum().nlargest(50)

acft_class
OTHER    1221786
HE        241854
PP        229673
PJ         59978
TP         49991
NB             1
Name: n_flights, dtype: int64

In [31]:
os_df[os_df.seymour_proxy!='zzz'].groupby('acft_class')['n_flights'].sum().nlargest(50)

acft_class
NB       6345337
WB       1349594
RJ       1287062
PP        660801
PJ        539243
TP        481820
OTHER       9671
Name: n_flights, dtype: int64

Now, let's compute the fuel burn using the surrogate.

In [32]:
fuel_surrogate = pd.read_csv('data/FuelSurrogate.csv', sep=';')
fuel_surrogate_dict=fuel_surrogate[['ac_code_icao',"reduced_fuel_intercept","reduced_fuel_a1","reduced_fuel_a2"]].set_index('ac_code_icao').transpose().to_dict()
ac_list=fuel_surrogate['ac_code_icao'].unique()


def compute_fuel_seymour(fuel_surrogate_dict, icao_type, distance):
    if icao_type in ac_list and distance > 1:
        coefficients = fuel_surrogate_dict[icao_type]
        fuel_burn = coefficients['reduced_fuel_intercept'] + coefficients['reduced_fuel_a2'] * distance + coefficients['reduced_fuel_a1'] * distance ** 2
    elif distance < 1:
        fuel_burn = 0
    else:
        fuel_burn = np.nan
    return float(fuel_burn)


os_df.loc[:, "fuel_burn_seymour"] = os_df.progress_apply(
                lambda x: compute_fuel_seymour(fuel_surrogate_dict, x["seymour_proxy"], x["distance_km"]),
                axis=1)

  0%|          | 0/856982 [00:00<?, ?it/s]

In [33]:
os_df

,iata_departure,departure_lon,departure_lat,departure_country,departure_continent,iata_arrival,arrival_lon,arrival_lat,arrival_country,arrival_continent,...,acft_class,seymour_proxy,seats,n_flights,source,airline_iata,distance_km,ask,rpk,fuel_burn_seymour
0,AAF,-85.027496,29.727501,US,NA,AAF,-85.027496,29.727501,US,NA,...,PP,zzz,0.0,1,OpenSky,ZZZ,0.000000,0.0,0.0,0.000000
1,AAF,-85.027496,29.727501,US,NA,AAF,-85.027496,29.727501,US,NA,...,OTHER,zzz,0.0,4,OpenSky,ZZZ,0.000000,0.0,0.0,0.000000
2,AAF,-85.027496,29.727501,US,NA,ALX,-85.962997,32.914700,US,NA,...,PP,P28B,0.0,1,OpenSky,ZZZ,364.420876,0.0,0.0,68.038310
3,AAF,-85.027496,29.727501,US,NA,APA,-104.848999,39.570099,US,NA,...,PP,zzz,0.0,1,OpenSky,ZZZ,2112.762830,0.0,0.0,NaN
4,AAF,-85.027496,29.727501,US,NA,CAE,-81.119499,33.938801,US,NA,...,PP,BE55,0.0,1,OpenSky,ZZZ,595.625664,0.0,0.0,221.362459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856977,ZZV,-81.892097,39.944401,US,NA,STL,-90.370003,38.748697,US,NA,...,PJ,C680,0.0,1,OpenSky,ZZZ,742.469955,0.0,0.0,991.822088
856978,ZZV,-81.892097,39.944401,US,NA,TBN,-92.140701,37.741600,US,NA,...,TP,PC12,0.0,1,OpenSky,ZZZ,922.144064,0.0,0.0,358.485500
856979,ZZV,-81.892097,39.944401,US,NA,TYS,-83.994003,35.811001,US,NA,...,PP,zzz,0.0,1,OpenSky,ZZZ,494.606225,0.0,0.0,NaN
856980,ZZV,-81.892097,39.944401,US,NA,TYS,-83.994003,35.811001,US,NA,...,PP,zzz,0.0,1,OpenSky,ZZZ,494.606225,0.0,0.0,NaN


In [34]:
os_df['fuel_burn']=os_df['fuel_burn_seymour']*os_df['n_flights']
os_df['co2']=os_df['fuel_burn']*3.16

os_df['domestic']=(os_df.departure_country == os_df.arrival_country).astype(int)

In [35]:
os_df.to_csv('data/os_alone_test.csv')